<a href="https://colab.research.google.com/github/BONHEMIA/CloudAppSena/blob/master/ETL_Tutorias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Montar Google Drive en Colab para acceder a los archivos.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Importar las bibliotecas necesarias.
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime
import re

In [ ]:
# Cargar el archivo CSV desde Google Drive.
tuto = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TESIS/StartTutorials.csv")

In [ ]:
# Carga la cantidad total de las filas y columnas.
tuto.shape

In [ ]:
# Identificar y mostrar el número de filas duplicadas
tuto[tuto.duplicated()].shape

In [ ]:
# Eliminar filas duplicadas y mostrar la nueva forma del DataFrame
tuto = tuto.drop_duplicates()
tuto.shape

In [ ]:
# Mostrar la cantidad de valores nulos por columna
tuto.isnull().sum()

In [ ]:
# Mostrar información general sobre el DataFrame
tuto.info()

In [ ]:
# Eliminar columnas  del DataFrame
tuto = tuto.drop(columns=['Actividad realizada', 'Tipo de tutoría', 'Vinculacion', 'Calificación estudiante'])

In [ ]:
# Función para separar números y letras con un espacio
def separar_numeros_letras(lugar):
    return ' '.join(filter(None, re.split(r'(\d+)', lugar)))

# Aplicar la función a la columna 'Lugar'
tuto['Lugar'] = tuto['Lugar'].apply(separar_numeros_letras)

# Muestra los resultados
print(tuto['Lugar'])

In [ ]:
# Asegúrate de que la columna 'lugar' esté en minúsculas para facilitar la comparación
tuto['Lugar'] = tuto['Lugar'].str.lower()

# Función para clasificar los valores
def clasificar_lugar(lugar):
    # Busca números en el lugar
    match = re.search(r'\b(\d+)\b', lugar)
    if match:
        numero = int(match.group(1))
        if 0 <= numero <= 399:
            return 'salón'
        elif 400 <= numero <= 499:
            return 'laboratorio'
        elif 500 <= numero <= 599:
            return 'oficina'

    # Otras clasificaciones por texto
    if 'salón' in lugar or 'salon' in lugar:
        return 'Salón'
    elif 'laboratorio' in lugar or 'laboratorios' in lugar:
        return 'Laboratorios'
    elif 'pecera' in lugar:
        return 'Pecera'
    elif 'zoom' in lugar or 'teams' in lugar or 'moodle' in lugar or 'link' in lugar or 'meet' in lugar or 'virtual' in lugar:
        return 'Mediación tecnológica'
    elif 'tutoria' in lugar or 'tutorías' in lugar or 'turia' in lugar or 'tutoría' in lugar:
        return 'sala de tutorías'
    elif 'seminario' in lugar or 'biblioteca' in lugar:
        return 'Biblioteca'
    elif 'consultorio jurídico' in lugar:
        return 'Consultorio jurídico'
    elif 'profesores' in lugar:
        return 'Sala de Profesores'
    else:
        return 'Otro'

# Crear la columna 'sitio' usando la función anterior
tuto['Sitio'] = tuto['Lugar'].apply(clasificar_lugar)

# Muestra los resultados
print(tuto[['Lugar', 'Sitio']])

In [ ]:
# Función para capitalizar palabras y asegurar que solo haya un espacio entre ellas
def limpiar_texto(columna):
    return columna.str.title().str.split().str.join(' ')

# Aplicar la función de limpieza a varias columnas del DataFrame
tuto['Facultad'] = limpiar_texto(tuto['Facultad'])
tuto['Programa'] = limpiar_texto(tuto['Programa'])
tuto['Nombre del curso'] = limpiar_texto(tuto['Nombre del curso'])
tuto['Temática'] = limpiar_texto(tuto['Temática'])
tuto['Modalidad'] = limpiar_texto(tuto['Modalidad'])
tuto['Metodología'] = limpiar_texto(tuto['Metodología'])
tuto['Lugar'] = limpiar_texto(tuto['Lugar'])
tuto['Nombre del grupo'] = limpiar_texto(tuto['Nombre del grupo'])
tuto['Acuerdos y compromisos'] = limpiar_texto(tuto['Acuerdos y compromisos'])
tuto['Asistencia'] = limpiar_texto(tuto['Asistencia'])
tuto['Comentarios'] = limpiar_texto(tuto['Comentarios'])
tuto['Sitio'] = limpiar_texto(tuto['Sitio'])

In [ ]:
# Especificar una fecha de reemplazo para los valores nulos
fecha_reemplazo = pd.to_datetime("01-01-1990 12:00", format="%d-%m-%Y %H:%M")

# Reemplazar valores nulos en las columnas de fecha con la fecha de reemplazo
tuto['Fecha de la tutoría'] = tuto['Fecha de la tutoría'].fillna(fecha_reemplazo)
tuto['Inicio de la Tutoría'] = tuto['Inicio de la Tutoría'].fillna(fecha_reemplazo)
tuto['Final de la Tutoría'] = tuto['Final de la Tutoría'].fillna(fecha_reemplazo)

In [ ]:
# Función para convertir fechas en diferentes formatos a un formato estándar ISO 8601
def convert_date(date_str):
    try:
        # Convertir formato '07-Mar-2023 10:00AM'
        return pd.to_datetime(date_str, format='%d-%b-%Y %I:%M%p').strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]
    except ValueError:
        # Convertir formato ISO 8601
        return pd.to_datetime(date_str).strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]

# Aplicar la conversión a la columna 'Fecha de la tutoría'
tuto['Fecha de la tutoría'] = tuto['Fecha de la tutoría'].apply(convert_date)
tuto['Fecha de la tutoría'] = pd.to_datetime(tuto['Fecha de la tutoría'])

# Crear columnas separadas para la fecha y la hora de la tutoría
tuto['FechaFechaTutoria'] = tuto['Fecha de la tutoría'].dt.date
tuto['HoraFechaTutoria'] = tuto['Fecha de la tutoría'].dt.time

# Eliminar la columna original 'Fecha de la tutoría'
tuto = tuto.drop(columns=['Fecha de la tutoría'])

# Repetir el proceso anterior para 'Inicio de la Tutoría'
tuto['Inicio de la Tutoría'] = tuto['Inicio de la Tutoría'].apply(convert_date)
tuto['Inicio de la Tutoría'] = pd.to_datetime(tuto['Inicio de la Tutoría'])

# Crear columnas separadas para la fecha y la hora de inicio de la tutoría
tuto['FechaIFechaTutoria'] = tuto['Inicio de la Tutoría'].dt.date
tuto['HoraIFechaTutoria'] = tuto['Inicio de la Tutoría'].dt.time

# Eliminar la columna original 'Inicio de la Tutoría'
tuto = tuto.drop(columns=['Inicio de la Tutoría'])

# Repetir el proceso anterior para 'Final de la Tutoría'
tuto['Final de la Tutoría'] = tuto['Final de la Tutoría'].apply(convert_date)
tuto['Final de la Tutoría'] = pd.to_datetime(tuto['Final de la Tutoría'])

# Crear columnas separadas para la fecha y la hora de finalización de la tutoría
tuto['FechaFFechaTutoria'] = tuto['Final de la Tutoría'].dt.date
tuto['HoraFFechaTutoria'] = tuto['Final de la Tutoría'].dt.time

# Eliminar la columna original 'Final de la Tutoría'
tuto = tuto.drop(columns=['Final de la Tutoría'])

In [ ]:
# Reemplazar valores nulos en la columna 'Acuerdos y compromisos' con su valor más común (moda)
moda_acuerdos = tuto['Acuerdos y compromisos'].mode()[0]
tuto['Acuerdos y compromisos'] = tuto['Acuerdos y compromisos'].fillna(moda_acuerdos)

In [ ]:
# Reemplazar valores nulos en la columna 'Comentarios' con su moda
moda_comentarios = tuto['Comentarios'].mode()[0]
tuto['Comentarios'] = tuto['Comentarios'].fillna(moda_comentarios)

In [ ]:
# Reemplazar valores nulos en la columna 'Calificación de la tutoría - estudiante' con su moda
moda_calificacion = tuto['Calificación de la tutoría - estudiante'].mode()[0]
tuto['Calificación de la tutoría - estudiante'] = tuto['Calificación de la tutoría - estudiante'].fillna(moda_calificacion)

In [ ]:
# Cambiar varios nombres de columnas a la vez
tuto = tuto.rename(columns={
    'Numero de sesión por estudiante': 'Sesion_Estudiante',
    'Nombre del curso': 'Curso',
    'Nombre del grupo': 'Grupo',
    'Documento del Profesor de la tutoria3': 'Documento_Profesor',
    'Estudiante': 'Documento_Estudiante',
    'FechaFechaTutoria': 'Fecha_Tutoria',
    'HoraFechaTutoria': 'Hora_tutoria',
    'FechaIFechaTutoria': 'Fecha_Inicio_Tutoria',
    'HoraFechaTutoria': 'Hora_tutoria',
    'HoraIFechaTutoria': 'Hora_Inicio_Tutoria',
    'FechaFFechaTutoria': 'Fecha_Fin_Tutoria',
    'HoraFFechaTutoria': 'Hora_Fin_Tutoria',
    'Calificación de la tutoría - estudiante': 'Calificacion_Tutoria',
    'Temática': 'Tematica',
    'Modalidad': 'Modalidad',
    'Metodología': 'Metodologia',
    'Lugar': 'Lugar',
    'Sitio': 'Sitio',
})

In [ ]:
print(tuto.columns)

In [ ]:
# Lista con el nuevo orden de las columnas
nuevo_orden = [
    'Fecha_Tutoria', 'Hora_tutoria', 'Fecha_Inicio_Tutoria', 'Hora_Inicio_Tutoria',
    'Fecha_Fin_Tutoria', 'Hora_Fin_Tutoria', 'Ciclo', 'Facultad', 'Programa',
    'Curso', 'Grupo', 'Tematica', 'Modalidad', 'Metodologia', 'Lugar', 'Sitio',
    'Documento_Profesor', 'Sesion_Estudiante', 'Documento_Estudiante', 'Acuerdos y compromisos',
    'Asistencia', 'Calificacion_Tutoria', 'Comentarios'
]

# Reordenar el DataFrame utilizando la lista de columnas
tuto = tuto[nuevo_orden]

In [ ]:
# Lista de columnas a convertir a valores absolutos
columnas_a_convertir = ['Documento_Profesor', 'Documento_Estudiante', 'Sesion_Estudiante', 'Calificacion_Tutoria']

# Convertir las columnas a valores absolutos solo si son numéricas
tuto[columnas_a_convertir] = tuto[columnas_a_convertir].apply(pd.to_numeric, errors='coerce').abs()

In [ ]:
# Guardar el DataFrame modificado en un nuevo archivo CSV en Google Drive
ruta_modificada = "/content/drive/MyDrive/Colab Notebooks/TESIS/StartTutorialsFinal1.csv"
tuto.to_csv(ruta_modificada, index=False)